In [ ]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import time
import re
from collections import OrderedDict

In [ ]:
dcard_food = pd.read_json('/Users/raymondyang/Desktop/dcard_project/clean_dcard_food0416.json')

In [ ]:
id_list = dcard_food['ID']
list1 = []
list2 = []
#len(id_list)

In [ ]:
start = time.time()
print('The program starts...')
count = len(id_list)
article_count = 0
for i in id_list:
    count = count -1
    print(count,'...')
    dcard_url = 'https://www.dcard.tw/f/food/p/' + str(i)
    response = requests.get(dcard_url)
    soup = BeautifulSoup(response.text)
    comment = soup.find_all(attrs={'CommentEntry_content_1ATrw1'})
    # Delete the duplications
    all_comment = list(set([j.text for j in comment]))
    #Assing a empty string to store the clean items from the list, and make all as a long list
    clean_comment = ''
    
    #Parse each element in the data list
    for item in all_comment:
        #clean img links
        item = re.sub('http(s?):([/|.|\w|\s|-])*\.(?:jpg|gif|png)','',item)
        #clean http(s) links
        item = re.sub(r'^(http:\/\/www\.|https:\/\/www\.|http:\/\/|https:\/\/)?[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,5}(:[0-9]{1,5})?(\/.*)?$','',item)
        #clean XD and so on
        item = re.sub('.~|^_^|￣^￣|XDD|ಠ_ಠ|๑´ڡ`๑*|｡･ω･｡|˶᷄ ̫ ᷅˵|;´༎ຶД༎ຶ`','',item)
        #clean '已經刪除的內容就像 Dcard 一樣，錯過是無法再相見的！'
        item = re.sub('已經刪除的內容就像 Dcard 一樣，錯過是無法再相見的！','',item)
        #clean any links
        item = re.sub('.http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+$','',item)
        #clean emojis
        RE_EMOJI = re.compile('(\u00a9|\u00ae|[\u2000-\u3300]|\ud83c[\ud000-\udfff]|\ud83d[\ud000-\udfff]|\ud83e[\ud000-\udfff]|[\U00010000-\U0010ffff])'
                              , flags = re.UNICODE)
        def strip_emoji(text):
            return RE_EMOJI.sub(r'', text)
        item=strip_emoji(item)
        clean_comment += item
        #id_comment_dict['Comment'] = clean_comment
    list1.append(clean_comment)
    list2.append(i)

data_parse = OrderedDict([('ID',list2),('Comment',list1)])
dcard_food_df = pd.DataFrame(data_parse)
dcard_food_df

with open('dcard_food_comment0416.json', 'w', encoding='utf-8') as file:
        dcard_food_df.to_json(file, force_ascii=False, orient='records')
end = time.time()
minute = round((end - start)/60)
second = round((end - start)%60)
#print(dcard_food_df.head())  # show the front rows
print('Finished')
print('Total time:',minute,'m',second,'s')

In [ ]:
# Read the file we just created, make sure every thing is correct
new_dcard_comment = pd.read_json('/Users/raymondyang/Desktop/dcard_project/dcard_food_comment0416.json')

In [ ]:
new_dcard_comment.head()

In [ ]:
new_dcard_comment.tail()

In [ ]:
len(new_dcard_comment)